<a href="https://colab.research.google.com/github/BuiThanhDam02/ML2023/blob/main/Lab_7_20130217_BuiThanhDam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This lab deals with **cross validation** for some classification algorithms and **clustering methods**. 

*   **Deadline: 23:59, 10/4/2023**



# Import libraries

In [62]:
# code
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score ,make_scorer,precision_score
from sklearn import metrics
import numpy as np
from scipy.stats import mode
import pandas as pd
from prettytable import  PrettyTable
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans

#Task 1. With **iris** dataset
*  1.1. Apply **SVM** algorithm with cross validation


In [40]:
#code
dataset = datasets.load_iris()
Xtrain = dataset['data']
ytrain = dataset['target']
clf = svm.SVC(kernel="sigmoid",random_state=0)
clf.fit(Xtrain, ytrain)
x_train, x_test, y_train, y_test = train_test_split(Xtrain,ytrain, test_size = 0.3, random_state = 0)


scores = cross_validate(clf,x_train,y_train,scoring='accuracy',cv=10)
print(sorted(scores.keys()))
print(np.mean(scores['test_score']))



['fit_time', 'score_time', 'test_score']
0.37272727272727274


In [22]:

cv= KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = svm.SVC (kernel='rbf', random_state=0)
# evaluate model
scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv)

print(scores.mean())

0.9627272727272727


*  1.2 Apply **feature selection** to the dataset and then use **RandomForest** algorithm with cross validation 

In [38]:
#code


rfc=RandomForestClassifier(n_estimators=100)

sfm = SelectFromModel(rfc, threshold=0.01)
Xtrain_new = sfm.fit_transform(Xtrain, ytrain)
rfc.fit(Xtrain_new, ytrain)
x_train, x_test, y_train, y_test = train_test_split(Xtrain_new,ytrain, test_size = 0.3, random_state = 0)

scores = cross_validate(rfc,x_train,y_train,scoring='accuracy',cv=10)

print(sorted(scores.keys()))
print(np.mean(scores['test_score']))
# rfc.fit(Xtrain_new,ytrain)

['fit_time', 'score_time', 'test_score']
0.9689650793650794


In [24]:

cv= KFold(n_splits=10, random_state=1, shuffle=True)

# evaluate model
scores = cross_val_score(rfc, x_train, y_train, scoring='accuracy', cv=cv)

print(scores.mean())

0.9518181818181818


#Task 2. With mnist dataset: 
*   2.1. Apply **K-Means** algorithm using k=10, 

In [64]:
# code
dataset = datasets.load_digits()
Xtrain = dataset['data']
ytrain = dataset['target']


kmeans = KMeans(n_clusters = 10, random_state = 0)
kmeans.fit(dataset.data)
clusters = kmeans.predict(dataset.data)
# kmeans.cluster_centers_.shape

labels = np.zeros_like(clusters)
for i in range(10):
 mask = (clusters == i)
 #print(mask)
 labels[mask] = mode(dataset.target[mask])[0]
 #print(labels[mask])
accKM = accuracy_score(dataset.target, labels)
preKM= metrics.precision_score(dataset.target, labels,average='macro')

recallKM= metrics.recall_score(dataset.target, labels,average='macro')
f1KM = metrics.f1_score(dataset.target, labels,average='macro') 
accKM


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-64-4737323bb15d>:16: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  labels[mask] = mode(dataset.target[mask])[0]


0.7935447968836951

*   2.2. Compare the obtained result with with other classification algorithms such as **Randomforest**, **kNN**, and **Naïve Bayes** in terms of accuracy, precision, recall, f1 using cross validation. 


In [66]:
# Random Forest

rfc=RandomForestClassifier(n_estimators=100)

rfc.fit(Xtrain,ytrain)
x_train, x_test, y_train, y_test = train_test_split(Xtrain,ytrain, test_size = 0.3, random_state = 0)

cv= KFold(n_splits=10, random_state=1, shuffle=True)

# evaluate model
scores = cross_val_score(rfc, x_train, y_train, scoring='accuracy', cv=cv)



y_predRFC = rfc.predict(x_test)

accRFC =  scores.mean()
preRFC= metrics.precision_score(y_test, y_predRFC,average='macro')

recallRFC= metrics.recall_score(y_test, y_predRFC,average='macro')
f1RFC = metrics.f1_score(y_test, y_predRFC,average='macro')

# GaussianNB Naive Bayes

nb = GaussianNB()
nb.fit(Xtrain, ytrain)


y_predNB = nb.predict(x_test)

scores = cross_val_score(nb, x_train, y_train, scoring='accuracy', cv=cv)

accNB =    scores.mean()
preNB= metrics.precision_score(y_test, y_predNB,average='macro')

recallNB= metrics.recall_score(y_test, y_predNB,average='macro')
f1NB = metrics.f1_score(y_test, y_predNB,average='macro') 

#Knn
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(Xtrain, ytrain)
y_predKnnBest = knn.predict(x_test)
    
scores = cross_val_score(knn, x_train, y_train, scoring='accuracy', cv=cv)

accKnn =  scores.mean()

preKnn = metrics.precision_score(y_test, y_predKnnBest,average='macro')
recallKnn= metrics.recall_score(y_test, y_predKnnBest,average='macro')
f1Knn = metrics.f1_score(y_test, y_predKnnBest,average='macro') 

#Table
t3 = PrettyTable(['Name Metric measure','KMean','Random Forest',"Naive Bayes (Gaussian)","Knn"])
t3.add_row(['Accuracy',accKM,accRFC,accNB,accKnn])
t3.add_row(['Precision Score',preKM,preRFC,preNB,preKnn])
t3.add_row(['Recall Score',recallKM,recallRFC,recallNB,recallKnn])
t3.add_row(['F1',f1KM,f1RFC,f1NB,f1Knn])

print(t3)

+---------------------+--------------------+--------------------+------------------------+--------------------+
| Name Metric measure |       KMean        |   Random Forest    | Naive Bayes (Gaussian) |        Knn         |
+---------------------+--------------------+--------------------+------------------------+--------------------+
|       Accuracy      | 0.7935447968836951 | 0.9705841269841271 |   0.8488444444444445   | 0.9824952380952382 |
|   Precision Score   | 0.8049510946090495 |        1.0         |   0.8754996278813421   | 0.9947880879546108 |
|     Recall Score    | 0.7935735538874538 |        1.0         |   0.8432015187160523   | 0.994736842105263  |
|          F1         | 0.7899801068577756 |        1.0         |   0.8469725015266443   | 0.994716022275721  |
+---------------------+--------------------+--------------------+------------------------+--------------------+


*   2.3. From the obtained results, **which approach is better** for this problem: Supervised learning or Unsupervised learning?

In [ ]:
#code


2.4. Apply **AgglomerativeClustering** algorithm to mnist dataset using the number of clusters is 10

#Task 3. 
For given dataset (shopping-data.csv) including 5 attributes: **CustomerID**, **Genre**, **Age**, **Annual Income**, and **Spending Score**.
*   3.1. Using the **scipy library** to create the dendrograms for the given dataset (remember drop categorical attributes: **CustomerID**, **Genre**)





In [ ]:
#code

*   3.2. Apply K-Means to the preprocessed dataset with k belongs to [2,10]. Then compute SSE values and plot them to find the best value of k.

In [ ]:
#code

*   3.2. From the obtained dengrograms, choose an appropriate number of clusters and apply **AgglomerativeClustering** algorithm to the given dataset

In [ ]:
#code

#Finally,
Save a copy in your Github. Remember renaming the notebook.